In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots

import statsmodels.api as sm

from statsmodels.stats.outliers_influence \
import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize,
poly)

In [ ]:
# dir() provides a list of objects in a namespace

In [ ]:
A = np.array([3,5,11])
# dir(A)

### Simple Linear Regression

In [ ]:
# We will construct model matrices / design matrices

In [ ]:
boston = pd.read_csv('Boston.csv')
boston.columns

In [ ]:
# boston?

In [ ]:
boston.shape

In [ ]:
boston.head(5)

In [ ]:
X = pd.DataFrame({'intercept': np.ones(boston.shape[0]),
                  'lstat': boston['lstat']}) # Creating model matrix by hand. 
# 1 single predictor (lstat)
X[:4]

In [ ]:
y = boston['medv']
model = sm.OLS(y, X)
results = model.fit()

In [ ]:
summarize(results) # ISLP function

In [ ]:
results.summary()

In [ ]:
results.params

#### More flexible way to construct a model matrix -> transform object

In [ ]:
design = MS(['lstat'])
design = design.fit(boston) # here the transformation occurs, as specified in the transform obj
X = design.transform(boston) # applies trans to the array of data and produces model matrix

# Better way is combining: X = design.fit_transform(Boston)

X[:4]

#### Make predictions

In [ ]:
new_df = pd.DataFrame({'lstat':[5, 10, 15]})
newX = design.transform(new_df)

predictions = results.get_prediction(newX);
predictions.predicted_mean

In [ ]:
predictions.conf_int(alpha=0.05)

##### Computing prediction intervals by setting obs=True

In [ ]:
predictions.conf_int(obs=True, alpha=0.05)
# Centered around the same point as CI but wider

In [ ]:
def abline(ax, b, m, *args, **kwargs):
    "Add a line with slope m and intercept b to ax"
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim)

In [ ]:
ax = boston.plot.scatter('lstat', 'medv')
abline(ax, results.params[0], results.params[1], 'r--', linewidth=3) ;
# 0 is b (intercept) while 1 is m (slope)

#### Diagnostic plots - is there evidence of non-linearity?

In [ ]:
ax = subplots(figsize=(8,8))[1]
ax.scatter(results.fittedvalues, results.resid) # residuals vs y^ or X is SLR
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--'); # horizontal black dashed line for reference

In [ ]:
infl = results.get_influence()
ax = subplots(figsize=(8,8))[1]

ax.scatter(np.arange(X.shape[0]), infl.hat_matrix_diag)
# X.shape[0] return 1st number in tuple returned by X.shape (aka no. of rows)
# 2nd arg (attribute of 'infl') computes leverage statistic for any number of predictors

ax.set_xlabel('Index')
ax.set_ylabel('Leverage')
np.argmax(infl.hat_matrix_diag) # Identify the index of the largest element of an array

### Multiple Linear Regression

In [ ]:
X = MS(['lstat', 'age']).fit_transform(boston) # construction of MM in 1 line.
model1 = sm.OLS(y, X)
results1 = model1.fit()
summarize(results1)

In [ ]:
# If p is very large
terms = boston.columns.drop('medv')
terms

In [ ]:
X = MS(terms).fit_transform(boston)
model = sm.OLS(y,X)
results = model.fit()
summarize(results)

#### Removing one predictor

In [ ]:
minus_age = boston.columns.drop(['medv', 'age'])
Xma = MS(minus_age).fit_transform(boston)
model1 = sm.OLS(y, Xma)
summarize(model1.fit())

In [ ]:
# dir(results)

In [ ]:
results.rsquared # R^2

In [ ]:
np.sqrt(results.scale) # RSE

#### Below, we compute the VIF for each feature in our X matrix and produce a data frame whose index agrees with the columns of X. The notion of list comprehension can often make such a task easier

In [ ]:
vals = [VIF(X, i)
        for i in range(1, X.shape[1])]
# VIF() takes a df/array and a variable column index

vif = pd.DataFrame({'VIF':vals},
                   index=X.columns[1:])
vif

### Including Interaction Terms

In [ ]:
X = MS(['lstat',
        'age',
        ('lstat', 'age')]).fit_transform(boston) # tuple signals interaction
model2 = sm.OLS(y, X)
summarize(model2.fit())

### Non-linear transformations of the predictors

In [ ]:
X = MS([poly('lstat', degree=2), 'age']).fit_transform(boston)
model3 = sm.OLS(y, X)
results3 = model3.fit()
summarize(results3)

In [ ]:
anova_lm(results1, results3) # further quantify the extent to which the quadratic fit is superior to the linear fit

##### anova_lm() performs a hyp test whose Ho is that the quadratic term in model3 is not needed. F-stat here is 177 and p-value=0.
##### F-stat is the square of the t-stat for the quadratic term in the linear model summary for results3 - a consequence of the fact that these nested models differ by 1 degree of freedom.
##### The function can take more than two inputs and compare every successive pair of models (explaining why the first row has NaNs).

In [ ]:
# I expect to find no pattern between residuals and the fitted values
ax = subplots(figsize=(8,8))[1]
ax.scatter(results3.fittedvalues , results3.resid)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--')

### Qualitative predictors

In [ ]:
Carseats = load_data('Carseats') # easier importing (ISLP module)
Carseats.columns

In [ ]:
Carseats.head(5)

#### One-hot encoding of the categorical feature -> MS() generates dummy variables automatically.
#### Their columns sum to one, so to avoid collinearity with an intercept, the first column is dropped (aka ShelveLoc[Bad] which corresponds to a zero of both Good and Medium)

In [ ]:
allvars = list(Carseats.columns.drop('Sales'))
y = Carseats['Sales']
final = allvars + [('Income', 'Advertising'),
                   ('Price', 'Age')] # Coding an interaction term

In [ ]:
X = MS(final).fit_transform(Carseats)
model = sm.OLS(y, X)
summarize(model.fit())